# File inputs and outputs
Suppose you want to read or write a text/binary file in your script, you will need the following functions to help you:
- open(*filename* \[, *access_mode*\]): Opens a file with the given *filename* under an access mode, e.g. "r" means read-only access, "w" means write-only access, and return the file object.

Syntax: 
```Python
fileObject = open(filename) # By default the access is read-only
```
- *fileObject*.read(*number*): Read the *number* of characters in the file (referenced by the file object).
- *fileObject*.readline( ): Read one line from the file and move to the next line.
- *fileObject*.write(*string*): Write the *string* to file.
- *fileObject*.close( ): Close the file.

These functions should have similar functionality in different languages, so you probably have learnt them before, and we will skip the details here.

# Working with NetCDF files

**In atmospheric science, we have a special data file format called NetCDF (.nc).** This specially designed data file format allows gridded atmospheric data to be saved easily with attributes and coordinates attached to each of the variables. However, we cannot read them using the standard way described above. The good news is someone has already written relevant modules to allow users to read/write .nc files relatively easily.

Before we can deal with those files, we need some extra packages (modules). New packages can be easily installed when creating a new conda environment. I have provided the [environment.txt](./environment.txt) file, which specifies a list of packages to be used. Follow the installation guide for [Mac](./pre_lab_installation_guide_mac.md#Install-conda-environment-for-extra-packages)/[Windows](./pre_lab_installation_guide_Windows.md#Install-conda-environment-for-extra-packages) to install them.

In [1]:
# import library to read .nc files
import xarray as xr
# You can check that this can only be done inside the environment, but not outside.

In [2]:
# Open the dataset using the library
ds = xr.open_dataset('./misc/sample_data.nc')
print(type(ds)) # Note that this is a class called Dataset
print(ds)

<class 'xarray.core.dataset.Dataset'>
<xarray.Dataset>
Dimensions:  (lat: 73, level: 4, lon: 144, time: 28)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    temp     (time, level, lat, lon) float32 ...
    hgt      (time, level, lat, lon) float32 ...
    uwnd     (time, level, lat, lon) float32 ...
    vwnd     (time, level, lat, lon) float32 ...
    tempsfc  (time, lat, lon) float32 ...
    slp      (time, lat, lon) float32 ...
    uwndsfc  (time, lat, lon) float32 ...
    vwndsfc  (time, lat, lon) float32 ...


Here we have just opened our data file, and `ds` is an xarray Dataset object. It contains all information of a NetCDF file. To use it, just think of `ds` as a dictionary, and apply the dictionary syntax (recall part 1):
```
value = dictionary['key']
```
where the *key* is the name of variable here.

## Extracting a variable
Let's get the temperature variable `temp` from the dataset. 

In [3]:
dr = ds['temp']
print(type(dr)) # Note that this is a class called DataArray
print(dr)

<class 'xarray.core.dataarray.DataArray'>
<xarray.DataArray 'temp' (time: 28, level: 4, lat: 73, lon: 144)>
[1177344 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:     4xDaily Air temperature
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    level_desc:    Multiple levels
    statistic:     Individual Obs
    parent_stat:   Other
    valid_range:   [150. 350.]
    dataset:       NCEP Reanalysis
    actual_range:  [195.69998 324.6    ]


`dr` is an xarray DataArray object, which behaves like a NumPy array. Therefore you can use NumPy syntax to operate on this array. However, it is functionally more than just NumPy array. You can see that there are **attributes** (sometimes called metadata) and **coordinates** attached to it. 

## Attributes
Suppose you want to access the attributes.

In [4]:
print(dr.attrs) 
# Note that these are attributes, not methods, so you don't need a pair of brackets after .attrs

OrderedDict([('long_name', '4xDaily Air temperature'), ('units', 'degK'), ('precision', 2), ('GRIB_id', 11), ('GRIB_name', 'TMP'), ('var_desc', 'Air temperature'), ('level_desc', 'Multiple levels'), ('statistic', 'Individual Obs'), ('parent_stat', 'Other'), ('valid_range', array([150., 350.], dtype=float32)), ('dataset', 'NCEP Reanalysis'), ('actual_range', array([195.69998, 324.6    ], dtype=float32))])


This is again a dictionary. Again you can use the dictionary syntax for accessing and modifying the attributes.

In [5]:
# Example for accessing attributes
print(dr.attrs['long_name'])

4xDaily Air temperature


In [6]:
# Example for adding new attribute
dr.attrs['new_attribute'] = 'This attribute has no meaning.'
print(dr.attrs)

OrderedDict([('long_name', '4xDaily Air temperature'), ('units', 'degK'), ('precision', 2), ('GRIB_id', 11), ('GRIB_name', 'TMP'), ('var_desc', 'Air temperature'), ('level_desc', 'Multiple levels'), ('statistic', 'Individual Obs'), ('parent_stat', 'Other'), ('valid_range', array([150., 350.], dtype=float32)), ('dataset', 'NCEP Reanalysis'), ('actual_range', array([195.69998, 324.6    ], dtype=float32)), ('new_attribute', 'This attribute has no meaning.')])


Suppose you want to change the unit from K to $^\circ$C. Then you will need to subtract the whole array by 273.15 and also change the *units* in attribute.

In [7]:
# Example for modifying attrbutes
dr_degC = dr - 273.15
print(dr_degC)

<xarray.DataArray 'temp' (time: 28, level: 4, lat: 73, lon: 144)>
array([[[[ -0.850006, ...,  -0.850006],
         ...,
         [-48.650024, ..., -48.650024]],

        ...,

        [[-48.250015, ..., -48.250015],
         ...,
         [-70.650024, ..., -70.650024]]],


       ...,


       [[[  3.549988, ...,   3.549988],
         ...,
         [-37.55002 , ..., -37.55002 ]],

        ...,

        [[-49.55002 , ..., -49.55002 ],
         ...,
         [-71.45001 , ..., -71.45001 ]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5


For whatever reason, the attributes won't be copied automatically to the new array. Let's do it manually

In [8]:
dr_degC.attrs = dr.attrs 
# Now we change the 'units' in attributes
dr_degC.attrs['units'] = 'degC'
print(dr_degC)
# You can see that the 'valid range' and 'actual range' should also be changed. 
# I changed the 'units' only for demonstration purpose.

<xarray.DataArray 'temp' (time: 28, level: 4, lat: 73, lon: 144)>
array([[[[ -0.850006, ...,  -0.850006],
         ...,
         [-48.650024, ..., -48.650024]],

        ...,

        [[-48.250015, ..., -48.250015],
         ...,
         [-70.650024, ..., -70.650024]]],


       ...,


       [[[  3.549988, ...,   3.549988],
         ...,
         [-37.55002 , ..., -37.55002 ]],

        ...,

        [[-49.55002 , ..., -49.55002 ],
         ...,
         [-71.45001 , ..., -71.45001 ]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:      4xDaily Air temperature
    units:          degC
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    level_d

## Coordinates
You may access the coordinates either from the xarray Dataset or DataArray Object, again using dictionary syntax.

In [9]:
print(ds.coords['lat'])

<xarray.DataArray 'lat' (lat: 73)>
array([ 90. ,  87.5,  85. ,  82.5,  80. ,  77.5,  75. ,  72.5,  70. ,  67.5,
        65. ,  62.5,  60. ,  57.5,  55. ,  52.5,  50. ,  47.5,  45. ,  42.5,
        40. ,  37.5,  35. ,  32.5,  30. ,  27.5,  25. ,  22.5,  20. ,  17.5,
        15. ,  12.5,  10. ,   7.5,   5. ,   2.5,   0. ,  -2.5,  -5. ,  -7.5,
       -10. , -12.5, -15. , -17.5, -20. , -22.5, -25. , -27.5, -30. , -32.5,
       -35. , -37.5, -40. , -42.5, -45. , -47.5, -50. , -52.5, -55. , -57.5,
       -60. , -62.5, -65. , -67.5, -70. , -72.5, -75. , -77.5, -80. , -82.5,
       -85. , -87.5, -90. ], dtype=float32)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
Attributes:
    units:          degrees_north
    long_name:      Latitude
    standard_name:  latitude
    axis:           Y
    actual_range:   [ 90. -90.]


In [10]:
print(dr.coords['lat'])

<xarray.DataArray 'lat' (lat: 73)>
array([ 90. ,  87.5,  85. ,  82.5,  80. ,  77.5,  75. ,  72.5,  70. ,  67.5,
        65. ,  62.5,  60. ,  57.5,  55. ,  52.5,  50. ,  47.5,  45. ,  42.5,
        40. ,  37.5,  35. ,  32.5,  30. ,  27.5,  25. ,  22.5,  20. ,  17.5,
        15. ,  12.5,  10. ,   7.5,   5. ,   2.5,   0. ,  -2.5,  -5. ,  -7.5,
       -10. , -12.5, -15. , -17.5, -20. , -22.5, -25. , -27.5, -30. , -32.5,
       -35. , -37.5, -40. , -42.5, -45. , -47.5, -50. , -52.5, -55. , -57.5,
       -60. , -62.5, -65. , -67.5, -70. , -72.5, -75. , -77.5, -80. , -82.5,
       -85. , -87.5, -90. ], dtype=float32)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
Attributes:
    units:          degrees_north
    long_name:      Latitude
    standard_name:  latitude
    axis:           Y
    actual_range:   [ 90. -90.]


In [11]:
# You may check to see if they are identical or not.
all(ds.coords['lat'] == dr.coords['lat']) 

True

And once again, each of the coordinates itself is an xarray DataArray object, so you may want to [revisit the syntax again](#Extracting-a-variable).

## Index slicing
As mentioned above, you can apply NumPy syntax, e.g. index slicing, on xarray DataArray objects.

In [12]:
# Let's look at the temperature data again
print(dr)

<xarray.DataArray 'temp' (time: 28, level: 4, lat: 73, lon: 144)>
array([[[[272.3    , ..., 272.3    ],
         ...,
         [224.49997, ..., 224.49997]],

        ...,

        [[224.89998, ..., 224.89998],
         ...,
         [202.49997, ..., 202.49997]]],


       ...,


       [[[276.69998, ..., 276.69998],
         ...,
         [235.59998, ..., 235.59998]],

        ...,

        [[223.59998, ..., 223.59998],
         ...,
         [201.69998, ..., 201.69998]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:      4xDaily Air temperature
    units:          degK
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    level_desc:     Multipl

Suppose you want the values for the lowest model level, in this case 1000hPa.

In [13]:
print(dr[:,0,:,:])

<xarray.DataArray 'temp' (time: 28, lat: 73, lon: 144)>
array([[[272.3    , 272.3    , ..., 272.3    , 272.3    ],
        [270.09998, 270.09998, ..., 270.09998, 270.09998],
        ...,
        [223.99997, 223.59998, ..., 224.59998, 224.29997],
        [224.49997, 224.49997, ..., 224.49997, 224.49997]],

       [[272.89996, 272.89996, ..., 272.89996, 272.89996],
        [270.89996, 270.89996, ..., 270.8    , 270.8    ],
        ...,
        [224.29997, 223.89998, ..., 225.09998, 224.69998],
        [225.09998, 225.09998, ..., 225.09998, 225.09998]],

       ...,

       [[275.7    , 275.7    , ..., 275.7    , 275.7    ],
        [273.9    , 273.9    , ..., 273.8    , 273.9    ],
        ...,
        [232.6    , 232.2    , ..., 233.4    , 233.     ],
        [237.5    , 237.5    , ..., 237.5    , 237.5    ]],

       [[276.69998, 276.69998, ..., 276.69998, 276.69998],
        [275.09998, 275.09998, ..., 275.09998, 275.09998],
        ...,
        [231.19998, 230.79997, ..., 231.99997, 

However, this is not very convenient if you don't know the coordinates at the first place (in other words, you need to know the coordinates first). 

Instead the xarray DataArray object offers a **grid-independent** *method* of index slicing. That means you only need to know what coordinates the array has (but not the order of coordinates), and you can grab a slice of the array that you need.

**Syntax**: (**SEL**ect by coordinate name and **I**ndex)
```
values = DataArrayObject.isel(coordinate_name = coordinate_index)
```
**Syntax**: (**SEL**ect by coordinate name and value)
```
values = DataArrayObject.sel(coordinate_name = coordinate_value)
```

**Warning**: However, according to the xarray documentation, you should **NEVER** assign values when using any of the indexing methods `isel` or `sel`. They fail silently.

In [14]:
# Again, let's choose the lowest model level
print(dr.isel(level=0))

<xarray.DataArray 'temp' (time: 28, lat: 73, lon: 144)>
array([[[272.3    , 272.3    , ..., 272.3    , 272.3    ],
        [270.09998, 270.09998, ..., 270.09998, 270.09998],
        ...,
        [223.99997, 223.59998, ..., 224.59998, 224.29997],
        [224.49997, 224.49997, ..., 224.49997, 224.49997]],

       [[272.89996, 272.89996, ..., 272.89996, 272.89996],
        [270.89996, 270.89996, ..., 270.8    , 270.8    ],
        ...,
        [224.29997, 223.89998, ..., 225.09998, 224.69998],
        [225.09998, 225.09998, ..., 225.09998, 225.09998]],

       ...,

       [[275.7    , 275.7    , ..., 275.7    , 275.7    ],
        [273.9    , 273.9    , ..., 273.8    , 273.9    ],
        ...,
        [232.6    , 232.2    , ..., 233.4    , 233.     ],
        [237.5    , 237.5    , ..., 237.5    , 237.5    ]],

       [[276.69998, 276.69998, ..., 276.69998, 276.69998],
        [275.09998, 275.09998, ..., 275.09998, 275.09998],
        ...,
        [231.19998, 230.79997, ..., 231.99997, 

In [15]:
# Get the data at 30N,60E
print(dr.sel(lat=30,lon=60))

<xarray.DataArray 'temp' (time: 28, level: 4)>
array([[301.69998, 300.2    , 266.9    , 226.19998],
       [314.39996, 302.39996, 266.49997, 227.     ],
       [320.39996, 308.3    , 266.09998, 225.8    ],
       [312.99997, 305.8    , 266.09998, 227.8    ],
       [302.7    , 300.8    , 266.     , 225.09998],
       [316.7    , 304.1    , 265.99997, 226.29997],
       [322.7    , 310.39996, 266.3    , 226.     ],
       [314.8    , 307.5    , 266.7    , 226.7    ],
       [304.2    , 301.59998, 266.2    , 225.     ],
       [316.6    , 304.2    , 266.59998, 224.7    ],
       [322.3    , 309.99997, 267.39996, 224.6    ],
       [313.5    , 306.6    , 268.2    , 225.2    ],
       [304.19998, 300.99997, 267.59998, 223.7    ],
       [317.19998, 304.39996, 267.39996, 224.4    ],
       [322.9    , 310.5    , 267.8    , 224.8    ],
       [314.8    , 307.69998, 268.3    , 225.3    ],
       [306.3    , 302.     , 267.69998, 224.8    ],
       [317.4    , 305.3    , 266.8    , 224.89998],

This is useful, but if you try other combinations, which the values may be absent from the coordinate array, then you will get an error. 

Fortunately, the developers have prepared for this situation. You can use an extra input argument `method='nearest'` to choose the data at the nearest coordinate.

In [16]:
print(dr.sel(lat=22.3,lon=114,method='nearest'))

<xarray.DataArray 'temp' (time: 28, level: 4)>
array([[299.8    , 292.9    , 269.3    , 221.89998],
       [302.99997, 294.19998, 269.3    , 223.     ],
       [301.19998, 294.3    , 269.69998, 222.8    ],
       [299.39996, 292.59998, 269.8    , 222.5    ],
       [299.4    , 292.5    , 269.4    , 222.39998],
       [301.5    , 293.1    , 269.89996, 223.29997],
       [301.5    , 294.8    , 271.     , 223.2    ],
       [298.7    , 292.6    , 270.4    , 223.3    ],
       [300.4    , 294.09998, 270.1    , 223.3    ],
       [302.5    , 294.8    , 270.19998, 223.7    ],
       [302.59998, 295.59998, 270.49997, 223.7    ],
       [300.     , 294.1    , 270.1    , 223.9    ],
       [299.8    , 293.3    , 270.39996, 223.5    ],
       [301.09998, 292.09998, 270.59998, 224.9    ],
       [301.4    , 293.1    , 270.99997, 224.9    ],
       [300.19998, 292.8    , 270.5    , 224.7    ],
       [300.39996, 292.7    , 269.69998, 224.1    ],
       [302.2    , 293.39996, 269.2    , 224.69998],

*Think more: How do you use the values of time coordinates to slice the array?*

Does these methods work with multiple indices/values? Let's try them.

In [17]:
# Well, this is the Pythonic way to do index slicing, so it should work very fine.
print(dr[:4,:,:36,:])

<xarray.DataArray 'temp' (time: 4, level: 4, lat: 36, lon: 144)>
array([[[[272.3    , ..., 272.3    ],
         ...,
         [299.09998, ..., 298.89996]],

        ...,

        [[224.89998, ..., 224.89998],
         ...,
         [220.79997, ..., 221.19998]]],


       ...,


       [[[274.49997, ..., 274.49997],
         ...,
         [299.59998, ..., 299.49997]],

        ...,

        [[221.7    , ..., 221.7    ],
         ...,
         [222.1    , ..., 222.3    ]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-09T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... 12.5 10.0 7.5 5.0 2.5
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:      4xDaily Air temperature
    units:          degK
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    level_desc:     Multiple l

Next, let's try the `isel` method:

In [18]:
# You may try using similar syntax, e.g. 0:4, as indices
# dr.isel(lat=0:3)
# However you will see that it does not work. 
# It's because 0:4 is not a valid syntax that gives a value for input arguments. (Note that isel is a method/function)
# Instead you need to provide a list (or something similar)
print(dr.isel(lat=range(36), time=range(4)))

<xarray.DataArray 'temp' (time: 4, level: 4, lat: 36, lon: 144)>
array([[[[272.3    , ..., 272.3    ],
         ...,
         [299.09998, ..., 298.89996]],

        ...,

        [[224.89998, ..., 224.89998],
         ...,
         [220.79997, ..., 221.19998]]],


       ...,


       [[[274.49997, ..., 274.49997],
         ...,
         [299.59998, ..., 299.49997]],

        ...,

        [[221.7    , ..., 221.7    ],
         ...,
         [222.1    , ..., 222.3    ]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-09T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... 12.5 10.0 7.5 5.0 2.5
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:      4xDaily Air temperature
    units:          degK
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    level_desc:     Multiple l

In [19]:
import numpy as np
print(dr.sel(lat=np.arange(31,61,2.5), lon=np.arange(0,180,10.0), method='nearest'))
# Because range() cannot deal with floats/decimal numbers, let's use the np.arange() instead. They work similarly.

<xarray.DataArray 'temp' (time: 28, level: 4, lat: 12, lon: 18)>
array([[[[304.8    , ..., 293.89996],
         ...,
         [285.39996, ..., 278.8    ]],

        ...,

        [[217.29997, ..., 222.19998],
         ...,
         [227.69998, ..., 229.59998]]],


       ...,


       [[[314.99997, ..., 296.89996],
         ...,
         [286.69998, ..., 279.3    ]],

        ...,

        [[218.79997, ..., 220.59998],
         ...,
         [232.99997, ..., 221.29997]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 30.0 32.5 35.0 37.5 40.0 ... 47.5 50.0 52.5 55.0 57.5
  * lon      (lon) float32 0.0 10.0 20.0 30.0 40.0 ... 140.0 150.0 160.0 170.0
Attributes:
    long_name:      4xDaily Air temperature
    units:          degK
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    level_desc:     Mult

*Think more: How to choose the steps in the `np.arange()`? What happens if the chosen step is too small? Note that the `sel()` method does not interpolate the array.*

One more way to use the `sel()` and `isel()` methods: Organize the indices using a dictionary.

In [20]:
# still remember how to initialize a dictionary? It is curly brackets {}
indices = {'lat':np.arange(30,60,2.5), 'lon':np.arange(0,180,10)}
print(indices)

{'lat': array([30. , 32.5, 35. , 37.5, 40. , 42.5, 45. , 47.5, 50. , 52.5, 55. ,
       57.5]), 'lon': array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170])}


In [21]:
print(dr.sel(indices, method='nearest'))
# Note that you cannot put method inside the dictionary.

<xarray.DataArray 'temp' (time: 28, level: 4, lat: 12, lon: 18)>
array([[[[304.8    , ..., 293.89996],
         ...,
         [285.39996, ..., 278.8    ]],

        ...,

        [[217.29997, ..., 222.19998],
         ...,
         [227.69998, ..., 229.59998]]],


       ...,


       [[[314.99997, ..., 296.89996],
         ...,
         [286.69998, ..., 279.3    ]],

        ...,

        [[218.79997, ..., 220.59998],
         ...,
         [232.99997, ..., 221.29997]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-06-09 ... 2017-06-15T18:00:00
  * level    (level) float32 1000.0 850.0 500.0 200.0
  * lat      (lat) float32 30.0 32.5 35.0 37.5 40.0 ... 47.5 50.0 52.5 55.0 57.5
  * lon      (lon) float32 0.0 10.0 20.0 30.0 40.0 ... 140.0 150.0 160.0 170.0
Attributes:
    long_name:      4xDaily Air temperature
    units:          degK
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    level_desc:     Mult

## Output to netCDF files
You may want to save the edited xarray DataArray object to a netCDF file.

**Syntax:**
```
DataArrayObject.to_netcdf(filename)
```

In [22]:
# Example
dr.sel(indices, method='nearest').to_netcdf('new_temp.nc')
# list all files to see the .nc you just saved
!ls # Windows users please replace this by !dir

Part0_Introduction.md                 README.md
Part1_Basic_Syntax.ipynb              environment.yml
Part1_Basic_Syntax.py                 images
Part2_Writing_Script.ipynb            misc
Part3_Flow_Control.ipynb              new_temp.nc
Part4_OOP.ipynb                       pre_lab_installation_guide_Windows.md
Part5_file_io.ipynb                   pre_lab_installation_guide_mac.md
Part6_plotting.ipynb
